In [1]:
import os
from os import listdir 
from os.path import isfile, join
import cv2

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Searching training data

label_folder = []
total_size = 0
data_path = r"C:\Users\ESA LAB\Desktop\Files\course\110-2\AI\03_final\resize_224"

#os.walk() generates the file names(dirpath, dirnames, filenames) 
#in a directory tree by walking the tree either top-down or bottom-up.
for root, dirts, files in os.walk(data_path): 
    for dirt in dirts:
        label_folder.append(dirt)
    
    total_size += len(files)

    
print("found",total_size,"files.")
print("folder:",label_folder)

found 80270 files.
folder: ['banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato']


In [3]:
#Load training images

base_x_train = []
base_y_train = []

for i in range(len(label_folder)):
    labelPath = data_path+r'\\'+label_folder[i]
    print(label_folder[i], end=' ')
    #listdir() returns a list containing the names of the entries in the directory given by path.
    #isfile() is used to check whether the specified path is an existing regular file or not.
    FileName = [f for f in listdir(labelPath) if isfile(join(labelPath, f))]
    
    for j in range(len(FileName)):
        path = labelPath+r'\\'+FileName[j]
        
        #use cv2.imread read image.
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        
        # img = img.astype('float')/255.0
        base_x_train.append(img)
        base_y_train.append(i)


print(np.array(base_x_train).shape)
print(np.array(base_y_train).shape)

banana bareland carrot corn dragonfruit garlic guava peanut pineapple pumpkin rice soybean sugarcane tomato (80270, 224, 224, 3)
(80270,)


In [4]:
base_x_train = np.array(base_x_train)
base_x_train = base_x_train.astype('float16')/255.0

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    base_x_train, base_y_train, test_size=0.2, random_state=42)

del base_x_train, base_y_train

In [6]:
import tensorflow as tf
import tensorflow_hub as hub 

In [7]:
mobile_net = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4' # MobileNetv4 link
mobile_net = hub.KerasLayer(
        mobile_net, input_shape=(224,224, 3), trainable=False)

In [8]:
num_label = 14 # number of labels

model = tf.keras.Sequential([
    mobile_net,
    tf.keras.layers.Dense(num_label)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 14)                17934     
Total params: 2,275,918
Trainable params: 17,934
Non-trainable params: 2,257,984
_________________________________________________________________


In [9]:
X_train_new, X_val, y_train_new, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

del X_train, y_train

In [10]:
from tensorflow.keras.utils import to_categorical

In [11]:
y_train_new = to_categorical(y_train_new)
y_val = to_categorical(y_val)

In [14]:
model.compile(
    loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=['acc']
)
history = model.fit(
    X_train_new,
    y_train_new,
    epochs=30,
    batch_size=20,
    validation_data=(X_val, y_val)
)


Epoch 1/30


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [32,14] and labels shape [448]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at C:\Users\ESALAB~1\AppData\Local\Temp/ipykernel_14308/2400575460.py:6) ]] [Op:__inference_train_function_22899]

Function call stack:
train_function


In [ ]:
model.save("my_model.hdf5")
model.save_weights("model.weight")

In [ ]:
prediction = model.predict(X_test)
prediction = np.argmax(prediction, axis = 1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction)*100

93.32876541671857

In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score

In [ ]:
model1 = load_model('my_model.hdf5')

In [ ]:
prediction1 = model1.predict(X_test)
prediction1 = np.argmax(prediction1, axis = 1)

In [ ]:
accuracy_score(y_test, prediction1)*100

93.32876541671857